In [1]:
from dask.distributed import Client, wait
from dask_cuda import DGX
from dask_cuda.initialize import initialize

In [2]:
enable_tcp_over_ucx = True
enable_infiniband = False
enable_nvlink = False

In [3]:
initialize(
    create_cuda_context=True,
    enable_tcp_over_ucx=enable_tcp_over_ucx,
    enable_infiniband=enable_infiniband,
    enable_nvlink=enable_nvlink,
)

In [4]:
from dask.distributed import Client, wait
from dask_cuda import DGX

# control number of workers with `CUDA_VISIBLE_DEVICES`
# definition
cluster = DGX(CUDA_VISIBLE_DEVICES=[0,1,2,3,4,5,6,7],
              enable_tcp_over_ucx=enable_tcp_over_ucx,
              enable_infiniband=enable_infiniband,
              enable_nvlink=enable_nvlink,
              dashboard_address=':8787')
client = Client(cluster);
client

distributed.dashboard.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:     ucx://127.0.0.1:35473
distributed.scheduler - INFO -   dashboard at:            127.0.0.1:8787
distributed.nanny - INFO -         Start Nanny at: 'ucx://127.0.0.1:47901'
distributed.nanny - INFO -         Start Nanny at: 'ucx://127.0.0.1:45269'
distributed.nanny - INFO -         Start Nanny at: 'ucx://127.0.0.1:52101'
distributed.nanny - INFO -         Start Nanny at: 'ucx://127.0.0.1:59295'
distributed.nanny - INFO -         Start Nanny at: 'ucx://127.0.0.1:54187'
distributed.nanny - INFO -         Start Nanny at: 'ucx://127.0.0.1:57395'
distributed.nanny - INFO -         Start Nanny at: 'ucx://127.0.0.1:44997'
distributed.nanny - INFO -         Start Nanny at: 'ucx://127.0.0.1:50141'
distributed.scheduler - INFO - Register ucx://127.0.0.1:

Client Scheduler: ucx://127.0.0.1:35473 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 1.08 TB


In [5]:
import dask_cudf, cudf, dask

In [6]:
import dask.array as da
import dask.dataframe as dd

n_rows = 1000000000
n_keys = 5000000

left = dd.concat([
    da.random.random(n_rows).to_dask_dataframe(columns='x'),
    da.random.randint(0, n_keys, size=n_rows).to_dask_dataframe(columns='id'),
], axis=1).persist()
left

,x,id
npartitions=64,,
0,float64,int64
15625000,...,...
...,...,...
984375000,...,...
999999999,...,...


In [7]:
n_rows = 10000000

right = dd.concat([
    da.random.random(n_rows).to_dask_dataframe(columns='y'),
    da.random.randint(0, n_keys, size=n_rows).to_dask_dataframe(columns='id'),
], axis=1).persist()
right

,y,id
npartitions=1,,
0,float64,int64
9999999,...,...


In [8]:
gleft = left.map_partitions(cudf.from_pandas)
gright = right.map_partitions(cudf.from_pandas)
# gright = gright.repartition(npartitions=8)
# gleft = gleft.repartition(npartitions=8)
gleft, gright = dask.persist(gleft, gright)  # persist data in device memory

In [9]:
from distributed.utils import format_bytes
right_gb = format_bytes(len(gright) * 8 * 2)
left_gb = format_bytes(len(gleft) * 8 * 2)
print(f'Total Left: {left_gb} Total Right: {right_gb}')

Total Left: 16.00 GB Total Right: 160.00 MB


In [10]:
out = gleft.merge(gright, on=['id'])  # this is lazy

In [11]:
out = out.persist()
%time _ = wait(out)

CPU times: user 752 ms, sys: 150 ms, total: 903 ms
Wall time: 10.1 s


In [12]:
out.head()

,x,id,y
0,0.059495,4715043,0.558606
1,0.228546,3600331,0.218486
2,0.038072,3742788,0.662214
3,0.914860,4125847,0.370489
4,0.743929,3312664,0.450289
